In [1]:
import os
os.environ['CMDSTAN'] = '/opt/anaconda3/envs/pangolin/bin/cmdstan'
from pycmdstan import Model, Run
import numpy as np

model = Model('''
data { vector[20] x; real mu; }
parameters { real sig; }
model {
    x ~ normal(mu, sig);
}
generated quantities {
    vector[20] log_lik;
    for (i in 1:20) log_lik[i] = normal_lpdf(x[i] | mu, sig);
}
''')

# mu = np.array([0.5])
# data = {}
#
# runs = model.sample(
#         data=dict(mu, **data),
#         chains=4
# )
# assert runs.N_eff_per_iter.min() > 0.2
# assert runs.R_hat.max() < 1.2

data = {'x': np.random.randn(20) + 5.0}
loo = []
mus = np.r_[1.0, 3.0, 5.0, 7.0, 9.0]
for mu in mus:
    run = model.sample(
        data=dict(mu=mu, **data), num_warmup=200, num_samples=200)
    loo.append(run['loo'])
assert mus[np.argmin(loo)] == 5.0

INFO:pycmdstan.model:starting /Users/j/.cache/pycmdstan/model-2777e6e1 id=68269 sample num_warmup=200 num_samples=200 data file=/var/folders/5z/ngs4vf717m12n9fd_sfbyyyw0000gn/T/tmp3_qrs_j1/data.R output file=/var/folders/5z/ngs4vf717m12n9fd_sfbyyyw0000gn/T/tmp3_qrs_j1/output.csv refresh=1


--- Translating Stan model to C++ code ---
bin/stanc  --o=/Users/j/.cache/pycmdstan/model-2777e6e1.hpp /Users/j/.cache/pycmdstan/model-2777e6e1.stan

--- Compiling the main object file. This might take up to a minute. ---
clang++ -std=c++1y -Wno-unknown-warning-option -Wno-tautological-compare -Wno-sign-compare -D_REENTRANT -Wno-ignored-attributes      -I /opt/anaconda3/envs/pangolin/include/    -O3 -I src -I stan/src -I stan/lib/rapidjson_1.1.0/ -I lib/CLI11-1.9.1/ -I stan/lib/stan_math/ -I stan/lib/stan_math/lib/eigen_3.3.9 -I stan/lib/stan_math/lib/boost_1.78.0 -I stan/lib/stan_math/lib/sundials_6.1.1/include -I stan/lib/stan_math/lib/sundials_6.1.1/src/sundials    -DBOOST_DISABLE_ASSERTS   -DTBB_INTERFACE_NEW  -DTBB_INTERFACE_NEW     -c -o src/cmdstan/main.o src/cmdstan/main.cpp
In file included from src/cmdstan/main.cpp:1:
In file included from src/cmdstan/command.hpp:4:
In file included from src/cmdstan/arguments/arg_data.hpp:4:
In file included from src/cmdstan/arguments/categor

FileNotFoundError: [Errno 2] No such file or directory: '/Users/j/.cache/pycmdstan/model-2777e6e1'

In [2]:
import multiprocessing
multiprocessing.set_start_method("fork")

import stan

schools_code = """
data {
  int<lower=0> J;         // number of schools
  real y[J];              // estimated treatment effects
  real<lower=0> sigma[J]; // standard error of effect estimates
}
parameters {
  real mu;                // population treatment effect
  real<lower=0> tau;      // standard deviation in treatment effects
  vector[J] eta;          // unscaled deviation from mu by school
}
transformed parameters {
  vector[J] theta = mu + tau * eta;        // school treatment effects
}
model {
  target += normal_lpdf(eta | 0, 1);       // prior log-density
  target += normal_lpdf(y | theta, sigma); // log-likelihood
}
"""

schools_data = {"J": 8,
                "y": [28,  8, -3,  7, -1,  1, 18, 12],
                "sigma": [15, 10, 16, 11,  9, 11, 10, 18]}

sm = stan.StanModel(model_code=schools_code)
fit = sm.sampling(data=schools_data, iter=1000, chains=4, seed=1)
fit.extract()["eta"]


AttributeError: module 'stan' has no attribute 'StanModel'

In [9]:
import nest_asyncio
nest_asyncio.apply()

import stan

schools_code = """
data {
  int<lower=0> J;         // number of schools
  array[J] real y;              // estimated treatment effects
  array[J] real<lower=0> sigma; // standard error of effect estimates
}
parameters {
  real mu;                // population treatment effect
  real<lower=0> tau;      // standard deviation in treatment effects
  vector[J] eta;          // unscaled deviation from mu by school
}
transformed parameters {
  vector[J] theta = mu + tau * eta;        // school treatment effects
}
model {
  target += normal_lpdf(eta | 0, 1);       // prior log-density
  target += normal_lpdf(y | theta, sigma); // log-likelihood
}
"""

schools_data = {"J": 8,
                "y": [28,  8, -3,  7, -1,  1, 18, 12],
                "sigma": [15, 10, 16, 11,  9, 11, 10, 18]}

posterior = stan.build(schools_code, data=schools_data)
fit = posterior.sample(num_chains=4, num_samples=1000)
eta = fit["eta"]  # array with shape (8, 4000)
df = fit.to_frame()  # pandas `DataFrame, requires pandas



Building...

INFO:aiohttp.access:127.0.0.1 [06/Nov/2023:21:26:39 +0000] "POST /v1/models/i4wqqx53/params HTTP/1.1" 404 250 "-" "Python/3.10 aiohttp/3.8.5"
INFO:httpstan:Building model-specific services extension module for `models/i4wqqx53`.
INFO:root:building 'stan_services_model_i4wqqx53' extension
INFO:root:clang -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /opt/anaconda3/envs/pangolin/include -fPIC -O2 -isystem /opt/anaconda3/envs/pangolin/include -DBOOST_DISABLE_ASSERTS -DBOOST_PHOENIX_NO_VARIADIC_EXPRESSION -DSTAN_THREADS -D_REENTRANT -D_GLIBCXX_USE_CXX11_ABI=0 -I/Users/j/Library/Caches/httpstan/4.10.0/models/i4wqqx53 -I/opt/anaconda3/envs/pangolin/lib/python3.10/site-packages/httpstan/include -I/opt/anaconda3/envs/pangolin/include/python3.10 -c /Users/j/Library/Caches/httpstan/4.10.0/models/i4wqqx53/model_i4wqqx53.cpp -o build/temp.macosx-10.9-x86_64-cpython-310/Users/j/Library/Caches/httpstan/4.10.0/models/i4wqqx53/model_i4wqqx53.o -O3 -

RuntimeError: Exception while building model extension module: `CompileError(DistutilsExecError("command '/opt/anaconda3/envs/pangolin/bin/clang' failed with exit code 1"))`, traceback: `['  File "/opt/anaconda3/envs/pangolin/lib/python3.10/site-packages/httpstan/views.py", line 114, in handle_create_model\n    compiler_output = await httpstan.models.build_services_extension_module(program_code)\n', '  File "/opt/anaconda3/envs/pangolin/lib/python3.10/site-packages/httpstan/models.py", line 172, in build_services_extension_module\n    compiler_output = await asyncio.get_running_loop().run_in_executor(\n', '  File "/opt/anaconda3/envs/pangolin/lib/python3.10/asyncio/futures.py", line 285, in __await__\n    yield self  # This tells Task to wait for completion.\n', '  File "/opt/anaconda3/envs/pangolin/lib/python3.10/asyncio/tasks.py", line 304, in __wakeup\n    future.result()\n', '  File "/opt/anaconda3/envs/pangolin/lib/python3.10/asyncio/futures.py", line 201, in result\n    raise sel

In [10]:
asyncio.run(asyncio.sleep(1))